# **AVANCE 3. BASELINE**

Nombre del Trabajo:
AI for Good: Human Heritage
Automatic pre-Hispanic to Spanish Recogniser and Translator


Nombre del asesor del proyecto: Dr. Juan Arturo Nolazco Flores

Equipo: 20
Nombre de alumnos:
 * Juan Carlos Garza Sanchez	A00821522
 * Sinaí Avalos Rivera 			A01730466

## **FEW-SHOTS LEARNING**

Este código hace todo el proceso de tomar una frase en Nahuatl, usar un ejemplo de traducción para generar una traducción automática a Español utilizando el modelo Gemini-Pro de Google y t5 large de hugging face. Luego evalúa esa traducción utilizando las métricas METEOR, BLEU y ROUGE. Escogimos estos modelos debido a que son de uso gratuito y son ampliamente utilizados para few shots learning.

Consideramos utilizar otros modelos, como mBART de Facebook por ejemplo, pero el problema de ellos es que necesitan más datos, toman más tiempo y además no estariamos haciendo few shots learning. Ya que el objetivo del proyecto es hacer traducción con few shots learning, se descartaron los modelos que no permiten few shots learning.

También descartamos el uso de OpenAI por el momento debido a que no es de uso gratuito.

In [4]:
!pip install google-cloud-aiplatform nltk==3.8.1 rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=8024f8825899751f495fcd8502b40be09a42b0ec3ea28406d1ed2a4fc3598806
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully uninstalled nltk-3.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.


In [5]:
import nltk
import re

# Recursos de NLTK para tokenizacion y WordNet
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer

import pandas as pd
import google.generativeai as genai

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [6]:
# Configurar la API de Gemini
GOOGLE_API_KEY = #Insert API KEY
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
# Cargar dataset
df = pd.read_csv("hf://datasets/somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl/train.csv")
df = df[['sp', 'nah']].rename(columns={'sp': 'spanish', 'nah': 'nahuatl'})
df = df.dropna(axis=0, how='any')
df = df.astype(str)
df

,spanish,nahuatl
0,"Y así, cuando hizo su ofrenda de fuego, se sie...",Auh in ye yuhqui in on tlenamacac niman ye ic ...
1,"¿Si es jade, si es oro, acaso no tendrá que ir...","¿In chalchihuitl, teocuitlatl, mach ah ca on yaz?"
2,"Y cuando el Sol estuvo solo en el cielo, enseg...",Auh yn oyuh in yoca hualmotlalli tonatiuh ylhu...
3,Allá acudieron asimismo los señores del cabild...,Yn oncan mohuicatza yhuan yn ciudad cabildo tl...
4,Usos:,Kualtia:
...,...,...
20023,"El Espíritu y la esposa dicen: ""¡Ven!""El que ...",On Espíritu Santo niman isihuau on Borreguito ...
20024,Yo advierto a todo el que oye las palabras de...,Nemechtlachicahuilia nenmochimej yejhuan nenqu...
20025,y si alguno quita de las palabras del libro d...,Niman tla yacaj quipopolohuilis itemachtil Dio...
20026,"El que da testimonio de estas cosas dice: ""¡S...","On yejhuan quipantlantia yejhua in, ijquin qui..."


In [8]:
# Seleccionar una frase de prueba para "One-Shot"
n = 20020
nth_row = df.iloc[[n]]
print(nth_row['nahuatl'].values[0])

Nemechtlachicahuilia nenmochimej yejhuan nenquicaquij itemachtil Dios yejhuan ipan in tlajcuilolamatl nesticaj: tla yacaj quimiyequilis in tlajtlamach, Dios quimiyequilis on plagas yejhuan ye tlajcuiloloncaj ipan in tlajcuilolamatl. 


In [9]:
# Seleccionar un solo ejemplo de entrenamiento (one-shot)
one_shot_example = df.iloc[[100]]


Definimos la función **create_one_shot_prompt** para generar un promt que instruye al modelo a traducir la frase correspondiente en Nahuatl a Español. Se da un ejemplo de traducción en el prompt y luego se solicita al modelo que traduzca la frase objetivo.

In [10]:
# Crear el prompt con un solo ejemplo
def create_one_shot_prompt(target_sentence, example):
    return f"""
    Translate the following Nahuatl sentence to Spanish:
    Example:
    "{example['nahuatl'].values[0]}" -> "{example['spanish'].values[0]}"

    Now, translate this sentence:
    "{target_sentence}"
    """

In [11]:
prompt = create_one_shot_prompt(nth_row['nahuatl'].values[0], one_shot_example)

In [12]:
def create_one_shot_prompt2(target_sentence, example):
    return f"""
    Translate the following Nahuatl sentence to Spanish:
    Nahuatl: "{example['nahuatl'].values[0]}"
    Spanish: "{example['spanish'].values[0]}"

    Nahuatl: "{target_sentence}"
    Spanish:
    """

In [13]:
prompt2 = create_one_shot_prompt2(nth_row['nahuatl'].values[0], one_shot_example)

### Usando Gemini

In [54]:
# Generar la traducción con Gemini
model = genai.GenerativeModel('gemini-pro')

In [17]:
response = model.generate_content(prompt)

Desplegar tanto la traducción generada por el modelo como la traducción esperada (de la fila seleccionada) para compararlas.

In [18]:
# Mostrar la traducción generada
print("Predicted:", response.text)
print("Expected:", nth_row['spanish'].values[0])

Predicted: Seamos reverentes con nuestros hijos y con los que reciben la enseñanza de Dios y en los libros de la escuela: si castigan mucho a los niños, Dios los castiga con plagas y con las enseñanzas escritas en libros.
Expected:  Yo advierto a todo el que oye las palabras de la profecía de este libro: Si alguno añade a estas cosas, Dios le añadirá las plagas que están escritas en este libro


Con la función normalize se toma el texto y lo convierte a minúsculas y elimina cualquier puntuación o caracteres no alfanuméricos, para ayudar a hacer las comparaciones de las métricas de manera más justa.

In [19]:
# Función para normalizar texto
def normalize(text):
    return re.sub(r'[^\w\s]', '', text.lower())  # Quita puntuación y convierte a minúsculas

In [20]:
# Normalizar referencia y predicción
reference = normalize(nth_row['spanish'].values[0])
hypothesis = normalize(response.text)

En este caso, estamos generando traducciones con un modelo preentrenado. Como este modelo no se entrena directamente, la preocupación de subajuste o sobreajuste se reduce, pero aún es posible evaluar la calidad de la traducción comparando los resultados obtenidos con la referencia.

Evaluación del desempeño: Dado que se está utilizando un modelo generativo, la comparación con las métricas como METEOR, BLEU, y ROUGE nos ayuda a detectar si el modelo está generando traducciones de baja calidad.

* METEOR score: Mide la calidad de las traducciones basándose en la correspondencia de n-gramas y sinónimos. El texto se tokeniza antes de calcular el score.
* BLEU score: Mide la calidad de la traducción usando n-gramas de diferentes tamaños. En este caso se utiliza el suavizado para ayudar a evitar la penalización excesiva por falta de n-gramas en las traducciones.
* ROUGE: Evalúa la calidad de la traducción en términos de precisión y recall a nivel de n-gramas y subsecuencias comúnes. 'Stemmer' ayuda a que las palabras se comparen en su forma raíz.

In [21]:
# Calcular METEOR score
meteor = meteor_score([word_tokenize(reference)], word_tokenize(hypothesis))
print(f"METEOR score: {meteor}")

METEOR score: 0.2567403108032203


In [66]:
# Calcular BLEU score con smoothing y BLEU-2
smoother = SmoothingFunction().method4
bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(hypothesis), weights=(0.5, 0.5), smoothing_function=smoother)
print(f"BLEU score: {bleu}")


BLEU score: 0.09239364353597955


In [24]:
# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference, hypothesis)
print(f"ROUGE scores: {rouge_scores}")


ROUGE scores: {'rouge1': Score(precision=0.30952380952380953, recall=0.3611111111111111, fmeasure=0.3333333333333333), 'rouge2': Score(precision=0.04878048780487805, recall=0.05714285714285714, fmeasure=0.05263157894736842), 'rougeL': Score(precision=0.2619047619047619, recall=0.3055555555555556, fmeasure=0.2820512820512821)}


En este caso, se puede observar que la traducción generada no coincide de forma esperada con la traducción esperada, lo cual es reflejado por las tres métricas de evaluación.

* **METEOR score: 0.21**
 * Este es un valor bajo, indicando que hay poca coincidencia en la correspondencia de n-gramas entre la traducción generada y la referencia. Por lo que, las traducciones podrían ser significativamente diferentes.
* **BLEU score: 0.12**
 * Tiene un valor bajo, lo que indica que la traducción generada no presenta una alta cantidad de n-gramas coincidentes con la traducción esperada.
* **ROUGE scores:**
 * **rouge1: Precision 0.3, Recall 0.33, F-measure 0.32**
 Este score mide la coincidencia de unigrams, teniendo un rendimiento moderado. Aunque se tienen algunas palabras coincidentes, la traducción generada no captura el significado de la referencia.
 * **rouge2: Precision 0.0, Recall 0.0, F-measure 0.0**
 No hay coincidencias de bigramas entre la referencia y lo obtenido, lo cual no es un buen indicador en la calidad de la traducción.
 * **rougeL: Precision 0.21, Recall 0.22, F-measure 0.21**
 Este score evalúa subsecuencias más largas, el valor obtenido muestra que hay un poco de correspondencia en las secuencias de palabras, pero no el esperado.


In [28]:
response = model.generate_content(prompt2)

In [29]:
# Mostrar la traducción generada
print("Predicted:", response.text)
print("Expected:", nth_row['spanish'].values[0])

Predicted: "Los hará saber a sus hijos y a los que vendrán después, aprendiendo del Libro de Dios y encontrando escrito lo siguiente: Si se rebela el pueblo, Dios desatará las plagas y entonces será escrito en el Libro. "
Expected:  Yo advierto a todo el que oye las palabras de la profecía de este libro: Si alguno añade a estas cosas, Dios le añadirá las plagas que están escritas en este libro


In [30]:
# Normalizar referencia y predicción
reference = normalize(nth_row['spanish'].values[0])
hypothesis = normalize(response.text)

In [31]:
# Calcular METEOR score
meteor = meteor_score([word_tokenize(reference)], word_tokenize(hypothesis))
print(f"METEOR score: {meteor}")

METEOR score: 0.22633350374914793


In [32]:
# Calcular BLEU score con smoothing y BLEU-2
smoother = SmoothingFunction().method4
bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(hypothesis), weights=(0.5, 0.5), smoothing_function=smoother)
print(f"BLEU score: {bleu}")

BLEU score: 0.09238425536915801


In [33]:
# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference, hypothesis)
print(f"ROUGE scores: {rouge_scores}")

ROUGE scores: {'rouge1': Score(precision=0.325, recall=0.3611111111111111, fmeasure=0.34210526315789475), 'rouge2': Score(precision=0.02564102564102564, recall=0.02857142857142857, fmeasure=0.027027027027027025), 'rougeL': Score(precision=0.225, recall=0.25, fmeasure=0.2368421052631579)}


En este caso, se puede observar que la traducción generada aún no coincide con lo esperado pero mejoro un poco comparado con el formato de prompt anterior.

* **METEOR score: 0.23**
 * Este es un valor bajo, indicando que hay poca coincidencia en la correspondencia de n-gramas entre la traducción generada y la referencia. Por lo que, las traducciones podrían ser significativamente diferentes.
* **BLEU score: 0.09**
 * Tiene un valor bajo, lo que indica que la traducción generada no presenta una alta cantidad de n-gramas coincidentes con la traducción esperada.
* **ROUGE scores:**
 * **rouge1: Precision 0.33, Recall 0.36, F-measure 0.34**
 Este score mide la coincidencia de unigrams, teniendo un rendimiento moderado. Aunque se tienen algunas palabras coincidentes, la traducción generada no captura el significado de la referencia.
 * **rouge2: Precision 0.03, Recall 0.03, F-measure 0.03**
 Hay muy pocas coincidencias de bigramas entre la referencia y lo obtenido, lo cual no es un buen indicador en la calidad de la traducción.
 * **rougeL: Precision 0.23, Recall 0.25, F-measure 0.24**
 Este score evalúa subsecuencias más largas, el valor obtenido muestra que hay un poco de correspondencia en las secuencias de palabras, pero no el esperado.

In [48]:
def create_prompt_with_examples(target_sentence, examples):
  prompt = "Translate the following Nahuatl sentences to Spanish:\n\n"
  for example in examples:
    prompt += f"*   \"{example['nahuatl'].values[0]}\" -> \"{example['spanish'].values[0]}\"\n"

  prompt += "\nNow, translate this sentence:\n\n"
  prompt += f"\"{target_sentence}\""
  return prompt

In [49]:
def create_prompt_with_examples2(target_sentence, examples):
  prompt = "Translate the following Nahuatl sentences to Spanish:\n\n"
  for example in examples:
    prompt += f"\nNahuatl: \"{example['nahuatl'].values[0]}\"  \nSpanish: \"{example['spanish'].values[0]}\"\n"
  prompt += f"\nNahuatl: \"{target_sentence}\""
  prompt += "\nSpanish:"
  return prompt

In [50]:
n = [20010, 20015, 20022]
shot1 = df.iloc[[n[0]]]
shot2 = df.iloc[[n[1]]]
shot3 = df.iloc[[n[2]]]

In [51]:
prompt = create_prompt_with_examples(nth_row["nahuatl"].values[0], [shot1, shot2, shot3])

In [52]:
prompt2 = create_prompt_with_examples2(nth_row["nahuatl"].values[0], [shot1, shot2, shot3])

In [55]:
response = model.generate_content(prompt)

In [56]:
print("Predicted:", response.text)
print("Expected:", nth_row['spanish'].values[0])

Predicted:  "Amenazo a aquellos que alteran mis palabras y las enseñanzas del Dios viviente registradas en este libro: si alguien suprime algo de estas enseñanzas, Dios le quitará su parte en el libro de la vida y del árbol de la vida "
Expected:  Yo advierto a todo el que oye las palabras de la profecía de este libro: Si alguno añade a estas cosas, Dios le añadirá las plagas que están escritas en este libro


In [57]:
reference = normalize(nth_row['spanish'].values[0])
hypothesis = normalize(response.text)

# Calcular METEOR score
meteor = meteor_score([word_tokenize(reference)], word_tokenize(hypothesis))
print(f"METEOR score: {meteor}")

# Calcular BLEU score con smoothing y BLEU-2
smoother = SmoothingFunction().method4
bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(hypothesis), weights=(0.5, 0.5), smoothing_function=smoother)
print(f"BLEU score: {bleu}")

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference, hypothesis)
print(f"ROUGE scores: {rouge_scores}")

METEOR score: 0.33923707154952093
BLEU score: 0.2208630521496931
ROUGE scores: {'rouge1': Score(precision=0.37209302325581395, recall=0.4444444444444444, fmeasure=0.40506329113924044), 'rouge2': Score(precision=0.11904761904761904, recall=0.14285714285714285, fmeasure=0.12987012987012989), 'rougeL': Score(precision=0.2558139534883721, recall=0.3055555555555556, fmeasure=0.27848101265822783)}


En este caso, se puede observar que la traducción generada aún no coincide con lo esperado pero mejoro un poco comparado con el formato de prompt anterior.

* **METEOR score: 0.34**
* **BLEU score: 0.22**
* **ROUGE scores:**
 * **rouge1: Precision 0.37, Recall 0.44, F-measure 0.40**
 * **rouge2: Precision 0.12, Recall 0.14, F-measure 0.13**
 * **rougeL: Precision 0.26, Recall 0.31, F-measure 0.28**.

In [63]:
response = model.generate_content(prompt2)

In [64]:
print("Predicted:", response.text)
print("Expected:", nth_row['spanish'].values[0])

Predicted: "Yo os felicito a los que me oís y guardáis estas palabras de la profecía. Porque el que las guarda, Dios le guardará de las plagas que están escritas en este libro."
Expected:  Yo advierto a todo el que oye las palabras de la profecía de este libro: Si alguno añade a estas cosas, Dios le añadirá las plagas que están escritas en este libro


In [65]:
reference = normalize(nth_row['spanish'].values[0])
hypothesis = normalize(response.text)

# Calcular METEOR score
meteor = meteor_score([word_tokenize(reference)], word_tokenize(hypothesis))
print(f"METEOR score: {meteor}")

# Calcular BLEU score con smoothing y BLEU-2
smoother = SmoothingFunction().method4
bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(hypothesis), weights=(0.5, 0.5), smoothing_function=smoother)
print(f"BLEU score: {bleu}")

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference, hypothesis)
print(f"ROUGE scores: {rouge_scores}")

METEOR score: 0.6304209183673469
BLEU score: 0.5040161287741853
ROUGE scores: {'rouge1': Score(precision=0.6388888888888888, recall=0.6388888888888888, fmeasure=0.6388888888888888), 'rouge2': Score(precision=0.4, recall=0.4, fmeasure=0.4000000000000001), 'rougeL': Score(precision=0.5277777777777778, recall=0.5277777777777778, fmeasure=0.5277777777777778)}


En este caso, se puede observar que la traducción generada esta mucho más cerca de lo esperado!

* **METEOR score: 0.63**
 * Este es el valor esperado, indicando que hay gran coincidencia en la correspondencia de n-gramas entre la traducción generada y la referencia. Por lo que, las traducciones podrían son similares.
* **BLEU score: 0.50**
 * Se acerca más al valor esperado pero no completamente, lo que indica que la traducción generada presenta una alta cantidad de n-gramas coincidentes con la traducción esperada.
* **ROUGE scores:**
 * **rouge1: Precision 0.64, Recall 0.64, F-measure 0.64**
 Este score mide la coincidencia de unigrams, teniendo un rendimiento moderado. Se tienen palabras que coinciden, la traducción generada captura la mayor parte del significado de la referencia.
 * **rouge2: Precision 0.4, Recall 0.4, F-measure 0.4**
 Hay coincidencias de bigramas entre la referencia y lo obtenido, lo cual es un buen indicador en la calidad de la traducción.
 * **rougeL: Precision 0.53, Recall 0.53, F-measure 0.53**
 Este score evalúa subsecuencias más largas, el valor obtenido se acerca ma´s al esperado y muestra que hay correspondencia en las secuencias de palabras.

### Prueba usando T5 de HuggingFace

In [34]:
!pip install --upgrade protobuf
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


In [66]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-large")
model = T5ForConditionalGeneration.from_pretrained("t5-large")

In [35]:
def translate(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

spanish_translation = translate(prompt)

In [36]:
# Mostrar la traducción generada
print("Predicted:", spanish_translation)
print("Expected:", nth_row['spanish'].values[0])

Predicted: yejhuan ye tlajcuilolamat
Expected:  Yo advierto a todo el que oye las palabras de la profecía de este libro: Si alguno añade a estas cosas, Dios le añadirá las plagas que están escritas en este libro


In [37]:
reference = normalize(nth_row['spanish'].values[0])
hypothesis = normalize(spanish_translation)

In [38]:
# Calcular METEOR score
meteor = meteor_score([word_tokenize(reference)], word_tokenize(hypothesis))
print(f"METEOR score: {meteor}")

METEOR score: 0.0


In [41]:
smoother = SmoothingFunction().method4
bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(hypothesis), weights=(0.5, 0.5), smoothing_function=smoother)
print(f"BLEU score: {bleu}")

BLEU score: 0


In [40]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference, hypothesis)
print(f"ROUGE scores: {rouge_scores}")

ROUGE scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}


En este caso, se puede observar que la traducción generada no coincide de forma esperada con la traducción esperada, lo cual es reflejado por las tres métricas de evaluación.

* **METEOR score: 0.0**
 * Este es un valor bajo, indicando que hay poca coincidencia en la correspondencia de n-gramas entre la traducción generada y la referencia. Por lo que, las traducciones podrían ser significativamente diferentes.
* **BLEU score: 0**
 * Tiene un valor bajo, lo que indica que la traducción generada no presenta una alta cantidad de n-gramas coincidentes con la traducción esperada.
* **ROUGE scores:**
 * **rouge1: Precision 0.0, Recall 0.0, F-measure 0.0**
 Este score mide la coincidencia de unigrams, teniendo un rendimiento moderado. No hay coincidencia.
 * **rouge2: Precision 0.0, Recall 0.0, F-measure 0.0**
 No hay coincidencias de bigramas entre la referencia y lo obtenido, lo cual no es un buen indicador en la calidad de la traducción.
 * **rougeL: Precision 0.0, Recall 0.0, F-measure 0.0**
 Este score evalúa subsecuencias más largas, el valor obtenido muestra que no hay correspondencia en las secuencias de palabras

In [42]:
spanish_translation = translate(prompt2)

In [43]:
print("Predicted:", spanish_translation)
print("Expected:", nth_row['spanish'].values[0])

Predicted: yejhuan ye tlajcuilolamat
Expected:  Yo advierto a todo el que oye las palabras de la profecía de este libro: Si alguno añade a estas cosas, Dios le añadirá las plagas que están escritas en este libro


In [44]:
reference = normalize(nth_row['spanish'].values[0])
hypothesis = normalize(spanish_translation)

In [45]:
meteor = meteor_score([word_tokenize(reference)], word_tokenize(hypothesis))
print(f"METEOR score: {meteor}")

METEOR score: 0.0


In [46]:
smoother = SmoothingFunction().method4
bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(hypothesis), weights=(0.5, 0.5), smoothing_function=smoother)
print(f"BLEU score: {bleu}")

BLEU score: 0


In [47]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference, hypothesis)
print(f"ROUGE scores: {rouge_scores}")

ROUGE scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}


In [67]:
spanish_translation = translate(prompt)

In [68]:
print("Predicted:", spanish_translation)
print("Expected:", nth_row['spanish'].values[0])

Predicted: tlajtlamach yejhuan yejh
Expected:  Yo advierto a todo el que oye las palabras de la profecía de este libro: Si alguno añade a estas cosas, Dios le añadirá las plagas que están escritas en este libro


In [69]:
reference = normalize(nth_row['spanish'].values[0])
hypothesis = normalize(spanish_translation)

# Calcular METEOR score
meteor = meteor_score([word_tokenize(reference)], word_tokenize(hypothesis))
print(f"METEOR score: {meteor}")

# Calcular BLEU score con smoothing y BLEU-2
smoother = SmoothingFunction().method4
bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(hypothesis), weights=(0.5, 0.5), smoothing_function=smoother)
print(f"BLEU score: {bleu}")

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference, hypothesis)
print(f"ROUGE scores: {rouge_scores}")

METEOR score: 0.0
BLEU score: 0
ROUGE scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}


In [70]:
spanish_translation = translate(prompt2)

In [71]:
print("Predicted:", spanish_translation)
print("Expected:", nth_row['spanish'].values[0])

Predicted: tlajtlamach yejhuan yejh
Expected:  Yo advierto a todo el que oye las palabras de la profecía de este libro: Si alguno añade a estas cosas, Dios le añadirá las plagas que están escritas en este libro


In [72]:
reference = normalize(nth_row['spanish'].values[0])
hypothesis = normalize(spanish_translation)

# Calcular METEOR score
meteor = meteor_score([word_tokenize(reference)], word_tokenize(hypothesis))
print(f"METEOR score: {meteor}")

# Calcular BLEU score con smoothing y BLEU-2
smoother = SmoothingFunction().method4
bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(hypothesis), weights=(0.5, 0.5), smoothing_function=smoother)
print(f"BLEU score: {bleu}")

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference, hypothesis)
print(f"ROUGE scores: {rouge_scores}")

METEOR score: 0.0
BLEU score: 0
ROUGE scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}


### Conclusion y siguientes pasos

Entre estos 2 modelos escogemos a Gemini debido a que produjo mejores resultados. Tambien vimos que al agregar más shots los scores mejoraron y se acercan a lo esperado. Cambiar el formato del prompt también cambio el score.

En el futuro podemos intentar con más shots y distintos formatos del prompt para ver como mejora.

En un futuro podemos volver a comparar con otros modelos de costo como openAI si tenemos los recursos para ello.